In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
im = Image.open('fake_samples_epoch_019.png')
image = np.array(im)
split_images = np.ones(shape=[64,32,32,3]).astype(image.dtype);
index = 0
for i in range(8):
    for j in range(8):
        split_images[index,:,:,:] = image[i*34+2:(i+1)*34,j*34+2:(j+1)*34,:]
        index = index+1

print split_images.shape
print image.shape

images = split_images.transpose([0,3,1,2])
# print split_images[2,:,:,:]
print images.shape
#split_images(64,32,32,3)

# print image[0*66+2:(0+1)*66,2*66+2:(2+1)*66,:]
plt.imshow(split_images[0,:,:,:]);
plt.show()
plt.imshow(image)
plt.show()



In [2]:
import torch
z = torch.randn(2500,100,1,1).cuda()

In [6]:
import torch.nn as nn
nz = 100
ngf = 64
nc = 3
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [1]:
nz = 100
ndf = 64
nc = 3
batch_size = 2500
bn = True
import torch.nn as nn
import torch.nn.functional as F
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.deconv1 = nn.ConvTranspose2d(nz,ndf*4,4,1,0,bias=False)
        self.bn1 = nn.BatchNorm2d(ndf*4)
        #[batch_size,ngf*4,4,4]
        self.deconv2 = nn.ConvTranspose2d(ndf*4,ndf*2,4,2,1,bias=False)
        self.bn2 = nn.BatchNorm2d(ndf*2)
        #[batch_size,ngf*2,8,8]
        self.deconv3 = nn.ConvTranspose2d(ndf*2,ndf*1,4,2,1,bias=False)
        self.bn3 = nn.BatchNorm2d(ndf)
        #[batch_size,ngf*1,16,16]
        self.deconv4 = nn.ConvTranspose2d(ndf,nc,4,2,1,bias=False)
        #[batch_size,3,32,32]
    def forward(self,x):
        #input x with shape[batch_size,nz,1,1]
        x = self.deconv1(x,output_size=[batch_size,ndf*4,4,4])
        if bn:
            x = self.bn1(x)
        x = F.relu(x)
        x = self.deconv2(x,output_size=[batch_size,ndf*2,8,8])
        if bn:
            x = self.bn2(x)
        x = F.relu(x)
        x = self.deconv3(x,output_size=[batch_size,ndf*1,16,16])
        if bn:
            x = self.bn3(x)
        x = F.relu(x)
        x = self.deconv4(x,output_size=[batch_size,nc,32,32])
        x = F.tanh(x)
        #return [batch_size,nc,32,32]
        return x

In [3]:
import torch

g_net = Generator()
g_net.load_state_dict(torch.load("gnet_090.pkl"))
g_net = g_net.cuda()
images = g_net(z)
print images.shape

torch.Size([2500, 3, 32, 32])


In [ ]:
import inception_score


In [17]:
print images[1,1,:,:]

tensor([[ 0.0835, -0.0147, -0.0750,  ...,  0.4951,  0.3799,  0.3156],
        [ 0.0191,  0.0234,  0.0581,  ...,  0.1702, -0.3978, -0.2674],
        [ 0.0236,  0.0684,  0.1433,  ..., -0.0899, -0.6069, -0.4634],
        ...,
        [ 0.1467,  0.1416,  0.1313,  ...,  0.2054,  0.1243,  0.1315],
        [ 0.1027,  0.1260,  0.1700,  ...,  0.0991,  0.0452,  0.1735],
        [ 0.0121,  0.0716,  0.1173,  ...,  0.2275,  0.2631,  0.2751]])


In [4]:

#inception_score(IgnoreLabelDataset(cifar), cuda=True, batch_size=32, resize=True, splits=10)
#def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
#   Computes the inception score of the generated images imgs
#     imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
#     cuda -- whether or not to run on GPU
#     batch_size -- batch size for feeding into Inception v3
#     splits -- number of splits
#     
import torch
import torch.utils.data as data
from inception_score import inception_score
class cifarDataset(data.Dataset):
    def __init__(self):
        self.data = images
    def __getitem__(self,index):
        return self.data[index]
    def __len__(self):
        return len(self.data)

if __name__=='__main__':
    dataset = cifarDataset()
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
    print (inception_score(dataset, cuda=True, batch_size=32, resize=True, splits=10))
#     for data in dataloader:
#         print data.shape
#         break
    #inception_score.inception_score(dataset,splits=10)
    
#gnet_050:4.96+0.15
#gnet_100:5.14+0.17    
#gnet_150:4.97+0.13    
    

/home/lrh/anaconda2/lib/python2.7/site-packages/torch/nn/functional.py:1749: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
inception_score.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


(4.6876891056295182, 0.14061890518570849)


In [4]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

if __name__ == '__main__':
    class IgnoreLabelDataset(torch.utils.data.Dataset):
        def __init__(self, orig):
            self.orig = orig

        def __getitem__(self, index):
            return self.orig[index][0]

        def __len__(self):
            return len(self.orig)

    import torchvision.datasets as dset
    import torchvision.transforms as transforms

    cifar = dset.CIFAR10(root='/home/lrh/dataset/cifar-10', download=False,
                             transform=transforms.Compose([
                                 transforms.Resize(32),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                             ])
    )

    IgnoreLabelDataset(cifar)
    dataloader = torch.utils.data.DataLoader(IgnoreLabelDataset(cifar), batch_size=32)
    
    for data in dataloader:
        print data.shape
        print data[1]
        break
    
    #print ("Calculating Inception Score...")
    #print (inception_score(IgnoreLabelDataset(cifar), cuda=True, batch_size=32, resize=True, splits=10))

torch.Size([32, 3, 32, 32])
tensor([[[ 0.2078, -0.0118, -0.1765,  ..., -0.2863, -0.3176, -0.3804],
         [ 0.0980,  0.1373, -0.0196,  ..., -0.2471, -0.3961, -0.4431],
         [ 0.0980,  0.0902, -0.0980,  ..., -0.3804, -0.4667, -0.4745],
         ...,
         [ 0.3725,  0.2235,  0.2078,  ..., -0.6706, -0.5216, -0.2706],
         [ 0.2941,  0.2235,  0.2471,  ..., -0.1922, -0.0353,  0.0275],
         [ 0.2784,  0.2392,  0.2784,  ...,  0.1216,  0.1216,  0.1216]],

        [[ 0.3882,  0.0745, -0.1843,  ..., -0.2549, -0.2941, -0.3647],
         [ 0.2549,  0.2000, -0.0196,  ..., -0.2235, -0.3725, -0.4275],
         [ 0.2157,  0.1451, -0.0980,  ..., -0.3569, -0.4510, -0.4588],
         ...,
         [ 0.3098,  0.2078,  0.2549,  ..., -0.7333, -0.5843, -0.3490],
         [ 0.2078,  0.1922,  0.2627,  ..., -0.2706, -0.1059, -0.0510],
         [ 0.1608,  0.1608,  0.2235,  ...,  0.0431,  0.0510,  0.0431]],

        [[ 0.4667,  0.0667, -0.2549,  ..., -0.4431, -0.4431, -0.4510],
         [ 0.3255